In [3]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from utils import *
from datetime import date

In [8]:
trainloader, testloader, classes = load_CIFAR()

Files already downloaded and verified
Files already downloaded and verified


### Batchnorm Training
- `model_bn` is the model with batchnorm layers
- train this model on the CIFAR 10 dataset

In [5]:
model_bn = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    nn.Tanh(),
    nn.BatchNorm1d(1152),
    nn.Linear(1152, 512),
    nn.Tanh(),
    nn.BatchNorm1d(512),
    nn.Linear(512, 128),
    nn.Tanh(),
    nn.BatchNorm1d(128),
    nn.Linear(128, 10)
)

In [7]:
loss_history_bn = []
writer_bn = SummaryWriter(log_dir='runs/bn_stats-%s' % date.today(), flush_secs=10)

In [15]:
ce_loss = nn.CrossEntropyLoss(reduction='sum')
def criterion(op, y, model_p):
    params = list(model_p.parameters())
    return ce_loss(op, y) #+ (1/len(params))*(torch.norm(torch.cat([param.view(-1) for param in params]))) #+ torch.sum((1/len(params))*torch.exp(torch.abs(torch.cat([param.view(-1) for param in params]))))


In [21]:
loss_tanh = train_model(model_bn, trainloader, criterion, optimizer = optim.Adam(list(model_bn.parameters()), lr=3e-4))

[1,   300] loss: 42.585
[1,   600] loss: 59.340
[2,   300] loss: 46.305
[2,   600] loss: 49.762
[3,   300] loss: 39.300
[3,   600] loss: 39.737
[4,   300] loss: 31.705
[4,   600] loss: 32.370
[5,   300] loss: 25.905
[5,   600] loss: 22.027


In [22]:
stats_bn = print_stats(model_bn)

Mean:  0.023430258
Standard Deviation:  0.02404121
Sparsity:  0.0028165338336380496


In [23]:
bn_model_err = get_model_error(model_bn, testloader)

test error: 0.3032
